In [1]:
#import necessary libraries for MNIST
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
sns.set()


In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.01 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

scaled_train_and_validation = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000
shuffled_train_and_validation = scaled_train_and_validation.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation.take(num_validation_samples)
train_data = shuffled_train_and_validation.skip(num_validation_samples)

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

MODEL

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(hidden_layer_size, activation='relu'),
    layers.Dense(hidden_layer_size, activation='relu'),
    layers.Dense(hidden_layer_size, activation='relu'),
    layers.Dense(hidden_layer_size, activation='relu'),
    layers.Dense(hidden_layer_size, activation='relu'),
    layers.Dense(output_size, activation='softmax')
])


c:\Users\USER\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


**OBJECTIVE FUNCTION**

Choosing the optimizer and the loss function

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**TRAINING THE MODEL**



In [6]:
num_epochs = 5
model.fit(train_data, epochs=num_epochs, validation_data=(validation_inputs, validation_targets), verbose=2)    


Epoch 1/5
594/594 - 14s - 23ms/step - accuracy: 0.9235 - loss: 0.2518 - val_accuracy: 0.9700 - val_loss: 0.1075
Epoch 2/5
594/594 - 8s - 13ms/step - accuracy: 0.9695 - loss: 0.0998 - val_accuracy: 0.9750 - val_loss: 0.0937
Epoch 3/5
594/594 - 7s - 12ms/step - accuracy: 0.9769 - loss: 0.0733 - val_accuracy: 0.9850 - val_loss: 0.0542
Epoch 4/5
594/594 - 8s - 13ms/step - accuracy: 0.9808 - loss: 0.0603 - val_accuracy: 0.9767 - val_loss: 0.0721
Epoch 5/5
594/594 - 7s - 12ms/step - accuracy: 0.9861 - loss: 0.0442 - val_accuracy: 0.9917 - val_loss: 0.0288


**TEST THE MODEL**

In [7]:
model.evaluate(test_data)
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}, Test Accuracy: {1:.2f}%'. format (test_loss, test_accuracy*100.))



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 852ms/step - accuracy: 0.9722 - loss: 0.1004
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step - accuracy: 0.9722 - loss: 0.1004
Test loss: 0.10, Test Accuracy: 97.22%
